In [1]:
import torch

### Tensor indexing

In [5]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [6]:
a[1]

tensor(1.)

In [7]:
torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])

tensor([4., 1., 5., 3., 2., 1.])

In [9]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [10]:
points.shape

torch.Size([3, 2])

In [11]:
# NOTE: indexing creates a new *view* of an existing tensor, which exists once in memory
points[0, 1]

tensor(1.)

In [13]:
points[0:2] # Slicing applies

tensor([[4., 1.],
        [5., 3.]])

In [14]:
points[1:, :] # All rows after the first; all columns

tensor([[5., 3.],
        [2., 1.]])

In [15]:
points[1:, 0] # All rows after the first; first column

tensor([5., 2.])

In [17]:
points[None].shape # Works like unsqueeze

torch.Size([1, 3, 2])

### Named tensors

In [18]:
# NOTE: this is still an experimental feature as shown below.

img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]

# Weights for the colours in the image for deriving a single brightness value.
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [19]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [20]:
# Target colour channels at index -3, regardless whether examples are batched or not
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [21]:
"""
PyTorch will allow us to multiply things that are the
same shape, as well as shapes where one operand is of size 1 in a given dimension.
This is "broarcasting".

Below, batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3,
1, 1), resulting in a tensor of shape (2, 3, 5, 5), from which we can then sum the third
dimension from the end (the three channels).
"""

unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [23]:
"""
The PyTorch function einsum (adapted from NumPy) specifies an indexing mini-language2 giving index
names to dimensions for sums of such products. As often in Python, broadcasting—a
form of summarizing unnamed things—is done using three dots '…'.
"""

img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [24]:
"""
However, as there is a lot of bookkeeping involved and potential for inconsistency in this,
PyTorch 1.3 added named tensors.
"""

weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

<ipython-input-24-b81c79f491ec>:6: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:840.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [25]:
"""
When we already have a tensor and want to add names (but not change existing
ones), we can call the method refine_names on it. Similar to indexing, the ellipsis (…)
allows you to leave out any number of dimensions.
"""

img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [26]:
# The method align_as returns a tensor with missing dimensions
# added and existing ones permuted to the right order

weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [27]:
# Functions accepting dimension arguments, like sum, also take named dimensions:

gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [28]:
# Combining dimensions with different names causes errors

gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [29]:
"""
If we want to use tensors outside functions that operate on named tensors, we need to
drop the names by renaming them to None. The following gets us back into the world
of unnamed dimensions
"""

gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [30]:
# Predicates on tensors produce bool tensors indicating whether each individual 
# element satisfies the condition

my_tens = torch.tensor([1, 2, 1])
my_tens <=1

tensor([ True, False,  True])

## Tensor d (data) type attribute

In [31]:
# Typical float tensors consist of 32-bit floating points (float32), 
# and integers 64-bit (int64), which can be used to index onto other tensors

double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [36]:
short_points.dtype

torch.int16

In [37]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

torch.int16

In [39]:
points_64 = torch.rand(5, dtype=torch.double) # rand = between 0 and 1
points_short = points_64.to(torch.short)
points_64 * points_short # works from PyTorch 1.3 onwards

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## Notable tensor operations

In [3]:
# Transpose (from torch and as method of tensor)
# Both methods are interchangeable

a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [4]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1) 

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

Other noteable tensor operations:

- Creation ops, such as *ones* and [*from_numpy*](https://pytorch.org/docs/master/generated/torch.from_numpy.html#torch-from-numpy)
- Indexing, slicing, joining, mutating ops
- Math ops (*abs*, *cos*, *mean*, *std*, *norm*, *equal*, *max*, [*stft*](https://pytorch.org/docs/stable/generated/torch.stft.html), [*hamming_window*](https://pytorch.org/docs/stable/generated/torch.hamming_window.html?highlight=hamming_window#torch.hamming_window))
- Special functions operating on vectors e.g. [*cross*](https://pytorch.org/docs/stable/generated/torch.cross.html?highlight=cross#torch.cross), and matrices e.g. [*trace*](https://pytorch.org/docs/stable/generated/torch.trace.html?highlight=trace#torch.trace)
- BLAS and LAPACK operations
    - BLAS = Basic Linear Algebra Subprograms
- Random sampling - *randn*, *normal*
- Serialization e.g. [*load*](https://pytorch.org/docs/master/generated/torch.load.html), [*save*](https://pytorch.org/docs/master/generated/torch.save.html)
- Parallelism e.g. [*set_num_threads*](https://pytorch.org/docs/master/generated/torch.set_num_threads.html)

## Tensors and storage

A PyTorch Tensor instance is a view of a torch.Storage instance that is capable of indexing
into that storage using an offset and per-dimension strides.

Given it's a view, multiple tensors can reference the same storage.

In [10]:
""" Storage is accessible using .storage
Note that even though the tensor 'points' reports itself as having three 
rows and two columns, the storage underneath is a contiguous array of size 6.
Layout of storage is always one-dimensional as such.
"""

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.shape

torch.Size([3, 2])

In [11]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [12]:
"""
Certain operations target storage directly and so allow us to operate on values in place
rather than simply create additional views/tensors.

They are characterized by a trailing underscore "_".
Any method *without* the trailing underscore will simply return a new tensor and will not
transform the underlying data of the source tensor.
"""

a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [13]:
"""
Key words in Storage

- Offset = the index in the storage corresponding to the first element in the tensor.
- Stride = the number of elements in the storage that need to be skipped
over to obtain the next element along each dimension
- Size = tuple of values representing the number of elements the storage holds for
each dimension of the tensor
"""

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [15]:
second_point.size(), second_point.shape

(torch.Size([2]), torch.Size([2]))

In [16]:
points.stride()

(2, 1)

NOTE: creation of a subtensor does not create new storage, but another view, so transforming
a subtensor will also modify the original tensor.

In [17]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

However, subtensors can be cloned into new tensors

In [18]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [19]:
# Transpose - inverts two dimensions to e.g. make columns rows and rows columns

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [20]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [21]:
# Verify whether two tensors share the same storage

id(points.storage()) == id(points_t.storage())

True

In [22]:
# Notice the differences in stride

points.stride(), points_t.stride()

((2, 1), (1, 2))

In [24]:
# The ability to transpose, given dimensions on which to do it, is not limited
# by a tensor's number of dimensions

some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [25]:
transpose_t.shape

torch.Size([5, 4, 3])

In [26]:
some_t.stride()

(20, 5, 1)

In [27]:
transpose_t.stride()

(1, 5, 20)

In [28]:
# Tensors which are contiguous can be visited efficiently in order without jumping around
# in storage

# Calls to contiguous() makes tensors contiguous, and has no overhead if called on a tensor
# which is already contiguous

points.is_contiguous()

True

In [29]:
points_t.is_contiguous()

False

In [31]:
# Making a tensor contiguous will change its stride and storage

points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [32]:
points_t.stride()

(1, 2)

In [34]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [35]:
points_t_cont.stride()

(3, 1)

In [36]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]